In [1]:
import os
from langchain_google_vertexai import ChatVertexAI
import subprocess

# os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"
os.environ["GOOGLE_API_KEY"] = subprocess.check_output(
    ["gcloud", "auth", "print-access-token"]).decode("utf-8").strip()
model = ChatVertexAI(model_name="gemini-2.0-flash-exp",
                     temperature=0.7,
                     location="us-central1")

In [2]:
from crewai.tools import tool
from crewai_tools import ScrapeWebsiteTool
import yfinance as yf


class Tools:

    @tool("One month stock price history")
    def stock_price(ticker: str):
        """
        Useful to get a month's worth of stock price data as CSV.
        The input of this tool should a ticker, for example "AAPL" or "TSLA".
        """
        stock = yf.Ticker(ticker)
        return stock.history(period="1mo").to_csv()

    @tool("Retrieve Stock news URLs")
    def stock_news(ticker: str):
        """
        Useful to get URLs of news articles related to a stock.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return list(map(lambda x: x["link"], stock.news))

    @tool("Company's income statement")
    def income_stmt(ticker: str):
        """
        Useful to get the income statement of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.income_stmt.to_csv()

    @tool("Balance sheet")
    def balance_sheet(ticker: str):
        """
        Useful to get the balance sheet of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.balance_sheet.to_csv()

    @tool("Get insider transactions")
    def insider_transactions(ticker: str):
        """
        Useful to get insider transactions of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.insider_transactions.to_csv()


In [3]:
from crewai import Agent


class Agents:

    def technical_analyst(self):
        return Agent(
            role="Technical Analyst",
            goal=
            "Analyses the movements of a stock and provides insights on trends, entry points, resistance and support levels.",
            backstory=
            "An expert in technical analysis, you're known for your ability to predict stock movements and trends based on historical data. You provide valuable insights to your customers.",
            verbose=True,
            tools=[
                Tools.stock_price,
            ],
            llm=model,
        )

    def researcher(self):
        return Agent(
            role="Researcher",
            goal=
            "Gathers, interprets and summarizes vasts amounts of data to provide a comprehensive overview of the sentiment and news surrounding a stock.",
            backstory=
            "You're skilled in gathering and interpreting data from various sources to give a complete picture of a stock's sentiment and news. You read each data source carefuly and extract the most important information. Your insights are crucial for making informed investment decisions.",
            verbose=True,
            tools=[
                Tools.stock_news,
                ScrapeWebsiteTool(),
            ],
            llm=model,
        )

    def financial_analyst(self):
        return Agent(
            role="Financial Analyst",
            goal=
            "Uses financial statements, insider trading data, and other financial metrics to evaluate a stock's financial health and performance.",
            backstory=
            "You're a very experienced investment advisor who uses a combination of technical and fundamental analysis to provide strategic investment advice to your clients. You look at a company's financial health, market sentiment, and qualitative data to make informed recommendations.",
            verbose=True,
            tools=[
                Tools.balance_sheet,
                Tools.income_stmt,
                Tools.insider_transactions,
            ],
            llm=model,
        )

    def hedge_fund_manager(self):
        return Agent(
            role="Hedge Fund Manager",
            goal=
            "Manages a portfolio of stocks and makes strategic investment decisions to maximize returns using insights from financial analysts, technical analysts, and researchers.",
            backstory=
            "You're a seasoned hedge fund manager with a proven track record of making profitable investment decisions. You're known for your ability to manage risk and maximize returns for your clients.",
            verbose=True,
            llm=model,
        )

In [4]:
from crewai import Task


class Tasks:

    def research(self, agent):
        return Task(
            description=
            "Gather and analyze the latest news and market sentiment surrounding the stock of {company}. Provide a summary of the news and any notable shifts in market sentiment.",
            expected_output=
            f"Your final answer MUST be a detailed summary of the news and market sentiment surrounding the stock. Include any notable shifts in market sentiment and provide insights on how these factors could impact the stock's performance.",
            agent=agent,
            output_file="stock_news.md",
        )

    def technical_analysis(self, agent):
        return Task(
            description=
            "Conduct a detailed technical analysis of the price movements of {company}'s stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets.",
            expected_output=
            f"Your final answer MUST be a detailed technical analysis report that includes key support and resistance levels, chart patterns, and technical indicators. Provide insights on potential entry points, price targets, and any other relevant information that could help your customer make informed investment decisions.",
            agent=agent,
            output_file="technical_analysis.md",
        )

    def financial_analysis(self, agent):
        return Task(
            description=
            "Analyze {company}'s financial statements, insider trading data, and other financial metrics to evaluate the stock's financial health and performance. Provide insights on the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            expected_output=
            f"Your final answer MUST be a detailed financial analysis report that includes insights on the company's financial health, performance, and valuation. Provide an overview of the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            agent=agent,
            output_file="financial_analysis.md",
        )

    def investment_recommendation(self, agent, context):
        return Task(
            description=
            "Based on the research, technical analysis, and financial analysis reports, provide a detailed investment recommendation for {company}'s stock. Include your analysis of the stock's potential risks and rewards, and provide a clear rationale for your recommendation.",
            expected_output=
            f"Your final answer MUST be a detailed investment recommendation report to BUY or SELL, Entry Point, and Price Target for the stock that includes your analysis of the stock's potential risks and rewards. Provide a clear rationale for your recommendation based on the research, technical analysis, and financial analysis reports.",
            agent=agent,
            context=context,
            output_file="investment_recommendation.md",
        )

In [5]:
from crewai import Crew
from crewai.process import Process
# from langchain_openai import ChatOpenAI
from chromadb.utils.embedding_functions import GoogleVertexEmbeddingFunction

agents = Agents()
tasks = Tasks()

researcher = agents.researcher()
technical_analyst = agents.technical_analyst()
financial_analyst = agents.financial_analyst()
hedge_fund_manager = agents.hedge_fund_manager()

research_task = tasks.research(researcher)
financial_task = tasks.financial_analysis(financial_analyst)
technical_task = tasks.technical_analysis(technical_analyst)
recommend_task = tasks.investment_recommendation(
    hedge_fund_manager, [research_task, financial_task, technical_task])

crew = Crew(agents=[financial_analyst, technical_analyst, hedge_fund_manager],
            tasks=[financial_task, technical_task, recommend_task],
            verbose=True,
            process=Process.sequential,
            memory=True,
            embedder={
                "provider": "vertexai",
                "config": {
                    "model": "text-embedding-005",
                    "project_id": "bio-llm",
                    "region": "asia-northeast3",
                    "api_key": os.getenv("GOOGLE_API_KEY")
                }
            })

result = crew.kickoff(inputs={"company": "Cloudflare"})

# Agent: Financial Analyst
## Task: Analyze Cloudflare's financial statements, insider trading data, and other financial metrics to evaluate the stock's financial health and performance. Provide insights on the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.


# Agent: Financial Analyst
## Using tool: Balance sheet
## Tool Input: 
"{\"ticker\": \"NET\"}"
## Tool Output: 
,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Treasury Shares Number,0.0,,,
Ordinary Shares Number,337532000.0,330086000.0,323612000.0,308640000.0
Share Issued,337532000.0,330086000.0,323612000.0,308640000.0
Net Debt,1196498000.0,1232014000.0,845217000.0,274380000.0
Total Debt,1435203000.0,1577091000.0,1293206000.0,428301000.0
Tangible Book Value,595436000.0,443434000.0,791014000.0,796973000.0
Invested Capital,2046409000.0,2060156000.0,1974792000.0,1200215000.0
Working Capital,1415993000.0,1490014000.0,